# Read the records from the Geom Test of Reciprocity

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from sys import argv
from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize

from scipy import signal


from pyaspect.project import *
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.write import _write_header
from pyaspect.specfemio.read import _read_headers
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *


import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

## Set Project Paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_out_dir}/tmp/TestProjects/CGFR_Test

In [ ]:
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','CGFR_Test')
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalGeometricTestProject')
fwd_project_fqp = os.path.join(projects_fqp,'ForwardGeometricTestProject')
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}

## Define bandpass (this is just a secondary test to sos filter) (taken from scipy recipies)

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y



## Read Reciprocal Project RecordHeader and load data

In [ ]:
def _load_data(self,dtype,sl=slice(None,None,None),scale=1.0,rfunc=None):
    
    if dtype != 'b':
        raise Exception('can only read binary type data for the time being')

    l_data_x = []
    l_data_y = []
    l_data_z = []
    for idx, row in self.stations_df.iterrows():
        fp_prefix = row['data_fqdn']
        fp = os.path.join(projects_fqp,fp_prefix) 
        match_fp = fp + '.*X[XYZEN].sem*'
        for filepath in glob.glob(match_fp):
            comp = filepath.split('.')[-2][-1]
            fname = filepath.split("/")[-1]
            if comp == 'X' or comp == 'E':
                l_data_x.append(scale*(np.fromfile(filepath, dtype=np.float32)[sl].astype(np.float64)))
            elif comp == 'Y' or comp == 'N':
                l_data_y.append(scale*(np.fromfile(filepath, dtype=np.float32)[sl].astype(np.float64)))
            elif comp == 'Z':
                l_data_z.append(scale*(np.fromfile(filepath, dtype=np.float32)[sl].astype(np.float64)))
            else:
                raise Exception(f'Could not find component: "{comp}"')
                
    df_ne = self.stations_df.index.get_level_values('eid').nunique()
    df_ns = self.stations_df.index.get_level_values('sid').nunique()
    df_ns = self.stations_df.index.get_level_values('sid').nunique()
    '''
    for eidx, edf in recip_record_h.stations_df.groupby(level='eid'):
        for sidx, sdf in edf.groupby(level='sid'):
            for tidx, tdf in sdf.groupby(level='trid'):
                for gidx, tdf in tdf.groupby(level='gid'):
    '''

    self.stations_df['comp_EX'] = l_data_x
    self.stations_df['comp_NY'] = l_data_y
    self.stations_df['comp_Z']  = l_data_z


In [ ]:
import glob
recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record_h = _read_headers(recip_record_fqp)

ne = recip_record_h.nevents
ns = recip_record_h.nsrc
print(f'ne:{ne}, ns:{ns}')

_load_data(recip_record_h,'b',scale=1.0,sl=slice(None,-10,None))

print(f'Recip Header:\n{recip_record_h}')



## function for computing the derivatives and applying the bandpass to reciprocal traces

In [ ]:
def calulate_spacial_derivative(tdf,eidx,sidx,tidx,g_p1,g_m1,sos,comp_key,coord_key):
    gidx_0  = pd.IndexSlice[eidx,sidx,tidx,0]
    gidx_p1 = pd.IndexSlice[eidx,sidx,tidx,g_p1]
    gidx_m1 = pd.IndexSlice[eidx,sidx,tidx,g_m1]
    df_0    = tdf.loc[gidx_0]
    df_p1   = tdf.loc[gidx_p1]
    df_m1   = tdf.loc[gidx_m1]
    data_p1 = signal.sosfilt(sos, df_p1[comp_key])
    data_m1 = signal.sosfilt(sos, df_m1[comp_key])
    c_p1    = df_p1[coord_key]
    c_m1    = df_m1[coord_key]
    c_0     = df_0[coord_key]
    delta   = 0.5*(c_p1 - c_m1)
    h       = 2.0*np.abs(delta)
    c       = c_m1 + delta
    
    assert h != 0
    assert c_0-c == 0
    
    h_scale  = 1/h
    mt_trace = h_scale*(data_p1 - data_m1)
    
    return mt_trace


## Similar to cell directly above, but calculate full 9D Greens Functions

In [ ]:
delta = 50
comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
coord_dict = {0:'lon_xc',1:'lat_yc',2:'depth'}
sos = signal.butter(3, [1,10], 'bp', fs=1000, output='sos') #USE for fwd and Recip
#sos = signal.butter(4, 10, 'lp', fs=1000, output='sos') #USE for fwd and Recip

ne = recip_record_h.nevents
ng = 9    #num sations
nc = 3    #num components (x=0,y=1,z=0)
nf = 3    #num force-directions (ex=0,ny=1,zup=2)
nd = 3    #num direction/derivatives (d_ex=0,d_ny=1,d_zup=2)
nt = 4096-10 #num samples in trace/greens function (hard coded for testing only)
rgf_table = np.zeros((ne,ng,nc,nf,nd,nt),dtype=np.float64)

src_df = recip_record_h.solutions_df
l_trace_latlon = []
l_event_latlon = []
for eidx, edf in recip_record_h.stations_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        jdx = (eidx,sidx)
        if sidx == 0:
            l_trace_latlon.append([jdx,src_df.loc[jdx,"lon_xc"],src_df.loc[jdx,"lat_yc"]])
        for tidx, tdf in sdf.groupby(level='trid'):
            idx = (eidx,sidx,tidx,0)
            if sidx == 0 and eidx == 0:
                l_event_latlon.append([idx,tdf.loc[idx,"lon_xc"],tdf.loc[idx,"lat_yc"]])
            for comp_key in comp_dict.keys():
                ie = tidx
                ig = eidx
                ic = comp_dict[comp_key]
                fi = sidx
                
                # dx=0,dy=1,dz=2
                d = delta
                for di in range(3):
                    coord_key = coord_dict[di]
                    ip1 = di+1     #coord + h
                    im1 = ip1 + 3  #coord - h
                    if di == 2:
                        tm1 = ip1
                        ip1 = im1
                        im1 = tm1
                        #d = -delta #FIXME: This is FUGLY!
                    rgf_table[ie,ig,ic,fi,di,:] = calulate_spacial_derivative(tdf,eidx,sidx,tidx,ip1,im1,sos,comp_key,coord_key)
                
#assert False               
print(f'l_event_latlon:')
for evn in l_event_latlon:
    print(evn)
print()
print(f'l_trace_latlon:')
for trc in l_trace_latlon:
    print(trc)

## Plot and compare table

## Read Forward Record and load data

In [ ]:
fwd_record_fqp = os.path.join(fwd_project_fqp,'pyheader.project_record')
fwd_record_h = _read_headers(fwd_record_fqp)

ne = fwd_record_h.nevents
ns = fwd_record_h.nsrc
print(f'ne:{ne}, ns:{ns}')

#_load_data(fwd_record_h,'b',scale=1E7,sl=slice(10,None,None))
_load_data(fwd_record_h,'b',scale=1.0,sl=slice(10,None,None))

print(f'Forward Record:\n{fwd_record_h}')

## Inspec moment tensors from Forward event. Will use thise for making Reciprocal Traces

In [ ]:
def make_moment_tensor(src_h):
    
    mrr = src_h['mrr']
    mtt = src_h['mtt']
    mpp = src_h['mpp']
    mrt = src_h['mrt']
    mrp = src_h['mrp']
    mtp = src_h['mtp']
    
    h_matrix = np.array([[mrr,mrt,mrp],[mrt,mtt,mtp],[mrp,mtp,mpp]])
    
    return MomentTensor(m_up_south_east=h_matrix)


#print(f'Forward Record Sources:\n{fwd_record_h.solutions_df}')
SrcHeader = fwd_record_h.solution_cls

d_fwd_src = {}
for eidx, edf in fwd_record_h.solutions_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        idx = pd.IndexSlice[eidx,sidx]
        src = SrcHeader.from_series(fwd_record_h.solutions_df.loc[idx])
        #print(src)
        #mag    = src.mw
        #strike = src.strike
        #dip    = src.dip
        #rake   = src.rake
        #mt = MomentTensor(mw=mag,strike=strike,dip=dip,rake=rake)
        mt = make_moment_tensor(src)
        print(mt)
        d_fwd_src[eidx] = mt
        #print(f'mt.aki_m6:\n{mt.aki_richards_m6()}')
        #print(f'header.m6:\n{src.mt}\n')

for key in d_fwd_src:
    print(d_fwd_src[key].m6_up_south_east())

## Derive the x,y, and z components for each event from the sorted reciprocity traces

In [ ]:
ntr = 9
ne  = 7
nc  = 3
nt  = 4096 - 10

cmb_traces = np.zeros((ne,ntr,nc,nt),dtype=np.float64)
for ie in range(ne):
    #print(f'mt:\n{d_fwd_src[ie]}')
    mw = d_fwd_src[ie].magnitude
    m0    = d_fwd_src[ie].moment
    #mt_arr = d_fwd_src[ie].m6_up_south_east()/m0
    mt_arr = d_fwd_src[ie].m6_up_south_east()
    wzz =  mt_arr[0] #mrr
    wyy =  mt_arr[1] #mtt
    wxx =  mt_arr[2] #mpp
    wyz = -mt_arr[3] #mrt
    wxz =  mt_arr[4] #mrp
    wxy = -mt_arr[5] #mtp
    
    #print(f'wuu:{wzz}, wnn:{wyy}, wee:{wxx}, wnu:{wyz}, weu:{wxz}, wen:{wxy}')
    print(f'Mw:{mw:.2f}, M0:{m0:.2f}, wzz:{wzz:.3f}, wyy:{wyy:.3f}, wee:{wxx:.3f}, wxy:{wxy:.3f}, wxz:{wxz:.3f}, wyz:{wyz:.3f}')
    
    for it in range(ntr):
        icomp = 0
        for comp_key in ['comp_EX','comp_NY','comp_Z']:
            
                                              #rgf_table[ie,it,ic,  fi ,di,:] 
            cmb_traces[ie,it,icomp,:] += wxx*1*rgf_table[ie,it, 0,icomp, 0,:] #Matrix: Mee
            cmb_traces[ie,it,icomp,:] += wyy*1*rgf_table[ie,it, 1,icomp, 1,:] #Matrix: Mnn
            cmb_traces[ie,it,icomp,:] += wzz*1*rgf_table[ie,it, 2,icomp, 2,:] #Matrix: Mzz
            
            #Matrix: M1/Mxy
            cmb_traces[ie,it,icomp,:] += wxy*1*rgf_table[ie,it, 1,icomp, 0,:]
            cmb_traces[ie,it,icomp,:] += wxy*1*rgf_table[ie,it, 0,icomp, 1,:]
            
            #Matrix: M2/Mxz
            cmb_traces[ie,it,icomp,:] += wxz*1*rgf_table[ie,it, 0,icomp, 2,:]
            cmb_traces[ie,it,icomp,:] += wxz*1*rgf_table[ie,it, 2,icomp, 0,:]
            
            #Matrix: M3/Myz
            cmb_traces[ie,it,icomp,:] += wyz*1*rgf_table[ie,it, 1,icomp, 2,:]
            cmb_traces[ie,it,icomp,:] += wyz*1*rgf_table[ie,it, 2,icomp, 1,:]
            
            icomp += 1
        

## bandpass the "forward" traces to match the bandpass of the reciprocal traces

In [ ]:
ne = 7
ntr = 9
nc = 3
nt = 4096 - 10
fwd_traces = np.zeros((ne,ntr,nc,nt))

for eidx, edf in fwd_record_h.stations_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        for tidx, tdf in sdf.groupby(level='trid'):
            idx = pd.IndexSlice[eidx,sidx,tidx,0]
            ic = 0
            for comp_key in ['comp_EX','comp_NY','comp_Z']:
                fwd_traces[eidx,tidx,ic,:] =  signal.sosfilt(sos, tdf.loc[idx,comp_key])
                ic += 1

## Plot the "forward" traces (black) on top of the reciprocal constructed traces (fat-blue)

In [ ]:
%matplotlib inline
#%matplotlib notebook
rcomp_dict = {0:'E/X',1:'N/Y',2:'Z'}

ne  = 7
ntr = 9
nc  = 3
nplt = ne*ntr*nc 
fig, axs = plt.subplots(nplt,1,figsize=(15,4*nplt))
fig.subplots_adjust(hspace=.75)

ip = 0
for ie in range(ne):
    for itr in range(ntr):
        for ic in range(nc):
            int_cmb_traces = 0.0001*np.cumsum(cmb_traces[ie,itr,ic,:].copy())
            #recip_max = np.max(np.abs(int_cmb_traces))
            #comp_traces = int_cmb_traces/recip_max
            comp_traces =  int_cmb_traces
            #axs[ip].plot(comp_traces,linewidth=2,linestyle='--',zorder=0,label='Recip')
            axs[ip].plot(comp_traces,color='gold',alpha=0.5,linestyle='-',linewidth=2,zorder=1,label='Recip')
            cmt_traces = fwd_traces[ie,itr,ic,:].copy()
            #cmt_traces /= np.max(np.abs(cmt_traces))
            #cmt_traces = np.cumsum(fwd_traces[ie,itr,ic,:].copy()) # if velocity
            #fwd_max = np.max(np.abs(cmt_traces))
            #cmt_traces /= fwd_max
            #print(f'fwd_max: {fwd_max}\nrecip_max: {recip_max}\nr/f: {recip_max/fwd_max}')
            '''
            div_traces = np.zeros_like(cmt_traces)
            for i in range(len(div_traces)):
                if cmt_traces[i] != 0:
                    div_traces[i] = comp_traces[i]/cmt_traces[i]
                else:
                    div_traces[i] = 1.0
            '''
            #axs[ip].plot(div_traces,color='orange',linewidth=2,zorder=0,label='CMT')
            axs[ip].plot(cmt_traces*2.3,color='lightblue',alpha=0.5,linewidth=5,zorder=0,label='CMT')
            axs[ip].set_title(f'Event:{ie}, Trace:{itr}, Comp:{rcomp_dict[ic]}')
            '''
            if itr == 3:
                print(f'Trace-3:\n{fwd_record_h[ie,0,itr,0]}')
            ''';
            ip += 1

#assert ip == nplt
plt.show()

assert False

In [ ]:
np.sqrt(6)

In [ ]:
np.sqrt(5)

In [ ]:
np.e

In [ ]:
2.3**2

In [ ]:
4/np.sqrt(2)

In [ ]:
np.sqrt()

In [ ]:
x = np.array([[2,0,0],[0,4,0],[0,0,6]])
s = 1/np.sqrt(2)
y = s*np.sqrt(np.sum(x*x))
print(y)
print(x/y)

In [ ]:
class _XYZ(object):

    def __init__(self,ex,ny,z):

        self.ex = ex
        self.ny = ny
        self.z  = z

    '''
    @property
    def ex(self):
        return self.ex

    @property
    def ny(self):
        return self.ny

    @property
    def z(self):
        return self.z
    '''
        
class Cdata(object):
    
    def __init__(self,ax,ay,az):
        
        self.ax = ax
        self.ay = ay
        self.az = az
        
    def __getitem__(self,islice):
        return _XYZ(self.ax[islice],self.ay[islice],self.az[islice])

    @property
    def ex(self):
        return self.ax

    @property
    def ny(self):
        return self.ay

    @property
    def z(self):
        return self.az
    

    
    
    
    
    
    
   





















    
    
    
idata = Cdata(np.arange(10,20), np.arange(20,30), np.arange(30,40))

print(f'{ idata[::-1].ex == idata.ex[::-1] }')
print(f'{ idata[5::2].ny == idata.ny[5::2] }')
print(f'{ idata[:8:-3].z == idata.z[:8:-3] }')

x = idata[::-1].ex.copy()
y = idata[::-1].ex.copy()
y[0] = -1
if all( x == y ):
    print('yep')
    print( x == y )
else:
    print('nope')
    
        